### 데이터 경로 지정하기

In [1]:
import os
from os.path import join

data_dir = os.getenv('HOME')+'/aiffel/Hackathon_01/data'

train_path = join(data_dir, 'sales_train.csv')
test_path = join(data_dir, 'test.csv') 
items_path = join(data_dir, 'items.csv') 
item_categories_path = join(data_dir, 'item_categories.csv') 
shops_path = join(data_dir, 'shops.csv') 

print(train_path)
print(test_path)
print(items_path)
print(item_categories_path)
print(shops_path)

/home/aiffel-dj41/aiffel/Hackathon_01/data/sales_train.csv
/home/aiffel-dj41/aiffel/Hackathon_01/data/test.csv
/home/aiffel-dj41/aiffel/Hackathon_01/data/items.csv
/home/aiffel-dj41/aiffel/Hackathon_01/data/item_categories.csv
/home/aiffel-dj41/aiffel/Hackathon_01/data/shops.csv


### 데이터 불러오기

In [2]:
import pandas as pd 

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)
items_data = pd.read_csv(items_path)
item_categories_data = pd.read_csv(item_categories_path)
shops_data = pd.read_csv(shops_path)

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
items = pd.read_csv(items_path)
item_categories = pd.read_csv(item_categories_path)
shops = pd.read_csv(shops_path)

In [3]:
print('train data shape : {}'.format(train.shape))
print('test data shape : {}'.format(test.shape))

train data shape : (2935849, 6)
test data shape : (214200, 3)


In [4]:
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


### train 데이터에서 test 데이터에 없는 상품과 가게 없애기

In [5]:
print('train data shape : {}'.format(train.shape))

test_shops = test.shop_id.unique()
test_items = test.item_id.unique()

train = train[train.shop_id.isin(test_shops)]
train = train[train.item_id.isin(test_items)]

print('train data shape : {}'.format(train.shape))

train data shape : (2935849, 6)
train data shape : (1224439, 6)


### 중복 데이터 제거하기

In [6]:
print('train data shape : {}'.format(train.shape))

train.drop_duplicates(inplace=True, ignore_index = True)

train[train.duplicated()]

print('train data shape : {}'.format(train.shape))


train data shape : (1224439, 6)
train data shape : (1224434, 6)


### item_cnt_month 컬럼 추가하기

In [7]:
group = train.groupby(['date_block_num', 'shop_id', 'item_id']).agg({'item_cnt_day': ['sum']})

group.head()

item_cnt_day
                                        sum
date_block_num shop_id item_id             
0              2       33               1.0
                       482              1.0
                       491              1.0
                       839              1.0
                       1007             3.0

In [8]:
group.columns = ['item_cnt_month']

group.head()

item_cnt_month
date_block_num shop_id item_id                
0              2       33                  1.0
                       482                 1.0
                       491                 1.0
                       839                 1.0
                       1007                3.0

In [9]:
group.reset_index(inplace=True)

group.head()

,date_block_num,shop_id,item_id,item_cnt_month
0,0,2,33,1.0
1,0,2,482,1.0
2,0,2,491,1.0
3,0,2,839,1.0
4,0,2,1007,3.0


In [10]:
train = pd.merge(train, group, how='left')

print(len(train))
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_cnt_month
0,02.01.2013,0,59,22154,999.0,1.0,1.0
1,03.01.2013,0,25,2574,399.0,2.0,13.0
2,05.01.2013,0,25,2574,399.0,1.0,13.0
3,07.01.2013,0,25,2574,399.0,1.0,13.0
4,08.01.2013,0,25,2574,399.0,2.0,13.0


In [11]:
import numpy as np

train['item_cnt_month'] = (train['item_cnt_month'].fillna(0).clip(0, 30).astype(np.float16))

train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_cnt_month
0,02.01.2013,0,59,22154,999.0,1.0,1.0
1,03.01.2013,0,25,2574,399.0,2.0,13.0
2,05.01.2013,0,25,2574,399.0,1.0,13.0
3,07.01.2013,0,25,2574,399.0,1.0,13.0
4,08.01.2013,0,25,2574,399.0,2.0,13.0


### year, month 컬럼 추가하기

In [12]:
year = train['date'].apply(lambda x : str(x[6:])).astype(int)
month = train['date'].apply(lambda x : str(x[3:5])).astype(int)

train = train.assign(year = year)
train = train.assign(month = month)

train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_cnt_month,year,month
0,02.01.2013,0,59,22154,999.0,1.0,1.0,2013,1
1,03.01.2013,0,25,2574,399.0,2.0,13.0,2013,1
2,05.01.2013,0,25,2574,399.0,1.0,13.0,2013,1
3,07.01.2013,0,25,2574,399.0,1.0,13.0,2013,1
4,08.01.2013,0,25,2574,399.0,2.0,13.0,2013,1
...,...,...,...,...,...,...,...,...,...
1224429,10.10.2015,33,25,7409,299.0,1.0,3.0,2015,10
1224430,09.10.2015,33,25,7460,299.0,1.0,3.0,2015,10
1224431,14.10.2015,33,25,7459,349.0,1.0,1.0,2015,10
1224432,22.10.2015,33,25,7440,299.0,1.0,1.0,2015,10


### city 컬럼 추가하기

In [13]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [14]:
city = shops["shop_name"].apply(lambda x: str.replace(x, '!', '')).apply(lambda x: x.split(' ')[0])

shops['city'] = pd.Categorical(city).codes

shops.drop('shop_name', axis=1, inplace=True)

shops.head()

,shop_id,city
0,0,29
1,1,29
2,2,0
3,3,1
4,4,2


In [15]:
print('train data shape : {}'.format(train.shape))

train = pd.merge(train, shops, on="shop_id", how="left")

print('train data shape : {}'.format(train.shape))

train data shape : (1224434, 9)
train data shape : (1224434, 10)


In [16]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_cnt_month,year,month,city
0,02.01.2013,0,59,22154,999.0,1.0,1.0,2013,1,30
1,03.01.2013,0,25,2574,399.0,2.0,13.0,2013,1,13
2,05.01.2013,0,25,2574,399.0,1.0,13.0,2013,1,13
3,07.01.2013,0,25,2574,399.0,1.0,13.0,2013,1,13
4,08.01.2013,0,25,2574,399.0,2.0,13.0,2013,1,13


### item_category_id1 컬럼 추가하기

In [17]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [18]:
items.drop('item_name', axis=1, inplace=True)

items.head()

,item_id,item_category_id
0,0,40
1,1,76
2,2,40
3,3,40
4,4,40


In [19]:
print('train data shape : {}'.format(train.shape))

train = pd.merge(train, items, on="item_id", how="left")

print('train data shape : {}'.format(train.shape))

train data shape : (1224434, 10)
train data shape : (1224434, 11)


In [20]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_cnt_month,year,month,city,item_category_id
0,02.01.2013,0,59,22154,999.0,1.0,1.0,2013,1,30,37
1,03.01.2013,0,25,2574,399.0,2.0,13.0,2013,1,13,55
2,05.01.2013,0,25,2574,399.0,1.0,13.0,2013,1,13,55
3,07.01.2013,0,25,2574,399.0,1.0,13.0,2013,1,13,55
4,08.01.2013,0,25,2574,399.0,2.0,13.0,2013,1,13,55


### item_category_id2 컬럼 추가하기

In [21]:
item_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [22]:
item_category_id2 = item_categories['item_category_name'].apply(lambda x: str(x).split(' ')[0])

item_categories["item_category_id2"] = pd.Categorical(item_category_id2).codes

item_categories.drop('item_category_name', axis=1, inplace=True)

item_categories.head()

,item_category_id,item_category_id2
0,0,0
1,1,1
2,2,1
3,3,1
4,4,1


In [23]:
print('train data shape : {}'.format(train.shape))

train = pd.merge(train, item_categories, on="item_category_id", how="left")

print('train data shape : {}'.format(train.shape))

train data shape : (1224434, 11)
train data shape : (1224434, 12)


In [24]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_cnt_month,year,month,city,item_category_id,item_category_id2
0,02.01.2013,0,59,22154,999.0,1.0,1.0,2013,1,30,37,7
1,03.01.2013,0,25,2574,399.0,2.0,13.0,2013,1,13,55,9
2,05.01.2013,0,25,2574,399.0,1.0,13.0,2013,1,13,55,9
3,07.01.2013,0,25,2574,399.0,1.0,13.0,2013,1,13,55,9
4,08.01.2013,0,25,2574,399.0,2.0,13.0,2013,1,13,55,9


In [25]:
train.rename(columns = {"item_category_id" : "item_category_id1"}, inplace=True)

train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_cnt_month,year,month,city,item_category_id1,item_category_id2
0,02.01.2013,0,59,22154,999.0,1.0,1.0,2013,1,30,37,7
1,03.01.2013,0,25,2574,399.0,2.0,13.0,2013,1,13,55,9
2,05.01.2013,0,25,2574,399.0,1.0,13.0,2013,1,13,55,9
3,07.01.2013,0,25,2574,399.0,1.0,13.0,2013,1,13,55,9
4,08.01.2013,0,25,2574,399.0,2.0,13.0,2013,1,13,55,9


### date 컬럼 삭제하기

In [26]:
train.drop('date', axis=1, inplace=True)

train.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_cnt_month,year,month,city,item_category_id1,item_category_id2
0,0,59,22154,999.0,1.0,1.0,2013,1,30,37,7
1,0,25,2574,399.0,2.0,13.0,2013,1,13,55,9
2,0,25,2574,399.0,1.0,13.0,2013,1,13,55,9
3,0,25,2574,399.0,1.0,13.0,2013,1,13,55,9
4,0,25,2574,399.0,2.0,13.0,2013,1,13,55,9


In [27]:
train.tail()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_cnt_month,year,month,city,item_category_id1,item_category_id2
1224429,33,25,7409,299.0,1.0,3.0,2015,10,13,55,9
1224430,33,25,7460,299.0,1.0,3.0,2015,10,13,55,9
1224431,33,25,7459,349.0,1.0,1.0,2015,10,13,55,9
1224432,33,25,7440,299.0,1.0,1.0,2015,10,13,57,9
1224433,33,25,7460,299.0,1.0,3.0,2015,10,13,55,9


### test 데이터 똑같은 형태로 만들어주기

In [28]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [29]:
# date_block_num, year, month 추가하기

test['date_block_num'] = 34
test['year'] = 2015
test['month'] = 11

test.head()

,ID,shop_id,item_id,date_block_num,year,month
0,0,5,5037,34,2015,11
1,1,5,5320,34,2015,11
2,2,5,5233,34,2015,11
3,3,5,5232,34,2015,11
4,4,5,5268,34,2015,11


In [30]:
# city 컬럼 추가하기

test = pd.merge(test, shops, on="shop_id", how="left")

test.head()

,ID,shop_id,item_id,date_block_num,year,month,city
0,0,5,5037,34,2015,11,3
1,1,5,5320,34,2015,11,3
2,2,5,5233,34,2015,11,3
3,3,5,5232,34,2015,11,3
4,4,5,5268,34,2015,11,3


In [31]:
# item_category_id 컬럼 추가하기

test = pd.merge(test, items, on="item_id", how="left")

test.head()

,ID,shop_id,item_id,date_block_num,year,month,city,item_category_id
0,0,5,5037,34,2015,11,3,19
1,1,5,5320,34,2015,11,3,55
2,2,5,5233,34,2015,11,3,19
3,3,5,5232,34,2015,11,3,23
4,4,5,5268,34,2015,11,3,20


In [32]:
# item_category_id2 컬럼 추가하기

test = pd.merge(test, item_categories, on="item_category_id", how="left")

test.head()

,ID,shop_id,item_id,date_block_num,year,month,city,item_category_id,item_category_id2
0,0,5,5037,34,2015,11,3,19,5
1,1,5,5320,34,2015,11,3,55,9
2,2,5,5233,34,2015,11,3,19,5
3,3,5,5232,34,2015,11,3,23,5
4,4,5,5268,34,2015,11,3,20,5


In [33]:
# 컬럼 변경하기

test.rename(columns = {"item_category_id" : "item_category_id1"}, inplace=True)

test.head()

,ID,shop_id,item_id,date_block_num,year,month,city,item_category_id1,item_category_id2
0,0,5,5037,34,2015,11,3,19,5
1,1,5,5320,34,2015,11,3,55,9
2,2,5,5233,34,2015,11,3,19,5
3,3,5,5232,34,2015,11,3,23,5
4,4,5,5268,34,2015,11,3,20,5


In [34]:
# ID 삭제하기

test.drop('ID', axis=1, inplace=True)

test.head()

,shop_id,item_id,date_block_num,year,month,city,item_category_id1,item_category_id2
0,5,5037,34,2015,11,3,19,5
1,5,5320,34,2015,11,3,55,9
2,5,5233,34,2015,11,3,19,5
3,5,5232,34,2015,11,3,23,5
4,5,5268,34,2015,11,3,20,5


### train 데이터와 test 데이터 확인하기

In [35]:
y = train["item_cnt_month"]

train.drop('item_cnt_day', axis=1, inplace=True)
train.drop('item_cnt_month', axis=1, inplace=True)
train.drop('item_price', axis=1, inplace=True)

print(train.shape)
print(train.isnull().sum())

train.head()

(1224434, 8)
date_block_num       0
shop_id              0
item_id              0
year                 0
month                0
city                 0
item_category_id1    0
item_category_id2    0
dtype: int64


,date_block_num,shop_id,item_id,year,month,city,item_category_id1,item_category_id2
0,0,59,22154,2013,1,30,37,7
1,0,25,2574,2013,1,13,55,9
2,0,25,2574,2013,1,13,55,9
3,0,25,2574,2013,1,13,55,9
4,0,25,2574,2013,1,13,55,9


In [36]:
print(test.shape)
print(test.isnull().sum())

test.head()

(214200, 8)
shop_id              0
item_id              0
date_block_num       0
year                 0
month                0
city                 0
item_category_id1    0
item_category_id2    0
dtype: int64


,shop_id,item_id,date_block_num,year,month,city,item_category_id1,item_category_id2
0,5,5037,34,2015,11,3,19,5
1,5,5320,34,2015,11,3,55,9
2,5,5233,34,2015,11,3,19,5
3,5,5232,34,2015,11,3,23,5
4,5,5268,34,2015,11,3,20,5


In [38]:
from lightgbm import LGBMRegressor

random_state=2021

lightgbm = LGBMRegressor(max_depth=10,
                         learning_rate=0.05,
                         num_iterations=1000,
                         boosting="gbdt",
                         random_state=random_state)
      
lightgbm.fit(train, y)

y_pred = lightgbm.predict(test)

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [39]:
y_pred

array([-4.45921753,  0.2487092 , -4.45921753, ..., -0.03434449,
        0.5135434 , -0.82575714])

In [40]:
test_data["item_cnt_month"] = y_pred

sub = test_data.loc[:, ["ID", "item_cnt_month"]]

sub

,ID,item_cnt_month
0,0,-4.459218
1,1,0.248709
2,2,-4.459218
3,3,-3.581143
4,4,1.014904
...,...,...
214195,214195,-0.034344
214196,214196,-0.054176
214197,214197,-0.034344
214198,214198,0.513543


In [41]:
sub_csv_path = data_dir + "/{}.csv".format("lightgbm")
sub.to_csv(sub_csv_path, index=False)
